# Predicting User Churn in Digital Music Services

Notebook to document data exploration and development of ML algorithm to identify at risk customers in digital music services.

### Data Definition

From Exploratory Data Analysis (EDA): 
#### Useful:
- *location*: location of user, seems to append each new state (location, state)
- *gender*: user gender (M/F/None)

- *page*: what page the user is on during event (pages)
- *level*: subscription level check uniqueness (free or paid)
- *auth*: authenication (logged in/out)
- *length*: time spent on page, max 50 mins on NextSong (if song paused??)

- *registration*: unknown (registration unixtime)
- *ts*: timestamp of event in ms (event unixtime)

- *userId*: unique (userId val)
- *sessionId*: unique sessionId per user?
- *itemInSession*: lcounter for the number of items in a single session (item listened to in session)


#### Not Useful:
- *firstName*: users first name (not important, remove)
- *lastName*: users lastname
- *artist*: song artist
- *song*: songname
- *userAgent*: device/browser (not important for us, remove)
- *method*: API PUT/GET http request (not important for us, remove)
- *status*: http status

# Apache Spark on IBM Watson Setup

### Imports

In [1]:
# imports
#import ibmos2spark

# pyspark sql
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import from_unixtime, udf, col, when, isnan, desc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import functions as F


# python
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

### setup

In [ ]:
# # config
# # @hidden_cell
# credentials = {
#     'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
#     'service_id': 'iam-ServiceId-147e1161-7da9-41fe-ac00-c144730def00',
#     'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
#     'api_key': 'kAtvjdC8VIYYUmU3gDaOYIK2fCvP3nkjYYlDiNuu4gw6'
# }

# configuration_name = 'os_76774389dfa04fb5acbb1640b3e11704_configs'
# cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [2]:
# Build Spark session
spark = SparkSession.builder.appName("user_churn").getOrCreate()

# Read in data from IBM Cloud
# data_df = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-fnqu5byx41gcai'))

data_df = spark.read.json("../data/01_raw/medium-sparkify-event-data.json")

# Exploratory Data Analysis

In [3]:
data_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [4]:
data_df.head(1)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293')]

In [5]:
data_df.toPandas().describe(include='all')

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
count,432877,543705,528005,528005,543705.000000,528005,432877.000000,543705,528005,543705,543705,5.280050e+05,543705.000000,432877,543705.000000,5.437050e+05,528005,543705
unique,21247,4,345,2,NaN,275,NaN,2,192,2,22,NaN,NaN,80292,NaN,NaN,71,449
top,Kings Of Leon,Logged In,Joseph,M,NaN,Reed,NaN,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,NaN,NaN,You're The One,NaN,NaN,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",
freq,3497,527906,13108,302612,NaN,12767,NaN,428597,40156,495143,432877,NaN,NaN,2219,NaN,NaN,46082,15700
mean,NaN,NaN,NaN,NaN,107.306291,NaN,248.664593,NaN,NaN,NaN,NaN,1.535523e+12,2040.814353,NaN,210.018291,1.540965e+12,NaN,NaN
std,NaN,NaN,NaN,NaN,116.723508,NaN,98.412670,NaN,NaN,NaN,NaN,3.078725e+09,1434.338931,NaN,31.471919,1.482057e+09,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,NaN,0.783220,NaN,NaN,NaN,NaN,1.509854e+12,1.000000,NaN,200.000000,1.538352e+12,NaN,NaN
25%,NaN,NaN,NaN,NaN,26.000000,NaN,199.392200,NaN,NaN,NaN,NaN,1.534368e+12,630.000000,NaN,200.000000,1.539720e+12,NaN,NaN
50%,NaN,NaN,NaN,NaN,68.000000,NaN,234.004440,NaN,NaN,NaN,NaN,1.536556e+12,1968.000000,NaN,200.000000,1.541005e+12,NaN,NaN
75%,NaN,NaN,NaN,NaN,147.000000,NaN,276.793020,NaN,NaN,NaN,NaN,1.537612e+12,3307.000000,NaN,200.000000,1.542177e+12,NaN,NaN


# ...

## Exploratory Data Analysis (EDA) -  using pysparksql

In [6]:
# create temp sql table to explore data
data_df.createOrReplaceTempView("user_log_table")

### Metadata: No. of Users in data

In [7]:
# how many users in the dataset, unique userId
spark.sql("SELECT COUNT(DISTINCT(userId)) FROM user_log_table LIMIT 10").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   449|
+----------------------+



### Feature: Types of Pages

In [22]:
# look at unique pages
spark.sql("SELECT DISTINCT(page) FROM user_log_table LIMIT 100").collect()

[Row(page='Cancel'),
 Row(page='Submit Downgrade'),
 Row(page='Thumbs Down'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Roll Advert'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Cancellation Confirmation'),
 Row(page='About'),
 Row(page='Submit Registration'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='Add to Playlist'),
 Row(page='Add Friend'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

From here we can see we want to identifying churned customers by using:
- "Cancellation Confirmation"

### Feature: Types of level

In [23]:
# unique levels
spark.sql("SELECT DISTINCT(level) FROM user_log_table LIMIT 100").show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



### Feature: authentication levels 

In [24]:
spark.sql("SELECT DISTINCT(auth) FROM user_log_table LIMIT 100").show()

+----------+
|      auth|
+----------+
|Logged Out|
| Cancelled|
|     Guest|
| Logged In|
+----------+



### Feature: User Locations

In [11]:
spark.sql("SELECT DISTINCT(location) FROM user_log_table LIMIT 1000").collect()

[Row(location='Gainesville, FL'),
 Row(location='Atlantic City-Hammonton, NJ'),
 Row(location='Richmond, VA'),
 Row(location='Tucson, AZ'),
 Row(location='Oskaloosa, IA'),
 Row(location='Deltona-Daytona Beach-Ormond Beach, FL'),
 Row(location='San Diego-Carlsbad, CA'),
 Row(location='Cleveland-Elyria, OH'),
 Row(location='Medford, OR'),
 Row(location='Kingsport-Bristol-Bristol, TN-VA'),
 Row(location='New Haven-Milford, CT'),
 Row(location='Birmingham-Hoover, AL'),
 Row(location='Corpus Christi, TX'),
 Row(location='Mobile, AL'),
 Row(location='Dubuque, IA'),
 Row(location='Las Vegas-Henderson-Paradise, NV'),
 Row(location='Killeen-Temple, TX'),
 Row(location='Ottawa-Peru, IL'),
 Row(location='Boise City, ID'),
 Row(location='Bremerton-Silverdale, WA'),
 Row(location='Urban Honolulu, HI'),
 Row(location='Cedar City, UT'),
 Row(location='Indianapolis-Carmel-Anderson, IN'),
 Row(location='Durham-Chapel Hill, NC'),
 Row(location='Seattle-Tacoma-Bellevue, WA'),
 Row(location='Fort Smith, A

In [12]:
# probably not worth cleaning
spark.sql("SELECT DISTINCT(userAgent) FROM user_log_table LIMIT 1000").collect()

[Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 Row(userAgent='"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"'),
 Row(userAgent='Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:31.0) Gecko/20100101 Firefox/31.0'),
 Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"'),
 Row(userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:31.0) Gecko/20100101 Firefox/31.0'),
 Row(userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0'),
 Row(userAgent='Mozilla/5.0 (Windows NT 6.3; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0'),
 Row(userAgent='Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW6

In [20]:
# probably not worth cleaning
spark.sql("SELECT gender, COUNT(*) FROM user_log_table GROUP BY gender LIMIT 1000").show()

+------+--------+
|gender|count(1)|
+------+--------+
|     F|  225393|
|  null|   15700|
|     M|  302612|
+------+--------+



In [27]:
# can see users changing from paid to free and vice versa
spark.sql("SELECT level, COUNT(DISTINCT(userId)) FROM user_log_table GROUP BY level LIMIT 1000").show()

+-----+----------------------+
|level|count(DISTINCT userId)|
+-----+----------------------+
| free|                   371|
| paid|                   322|
+-----+----------------------+



In [32]:
# cancellation events
spark.sql("SELECT COUNT(*) FROM user_log_table WHERE page='Cancellation Confirmation'").show()

+--------+
|count(1)|
+--------+
|      99|
+--------+



In [31]:
# total number of events
spark.sql("SELECT COUNT(*) FROM user_log_table").show()

+--------+
|count(1)|
+--------+
|  543705|
+--------+



In [33]:
# No. of churned users
spark.sql("SELECT COUNT(DISTINCT(userId)) FROM user_log_table WHERE page='Cancellation Confirmation'").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                    99|
+----------------------+

